## Libraries

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
from sklearn.metrics import r2_score, mean_absolute_error, mean_absolute_percentage_error
from pathlib import Path

In [ ]:
from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

In [ ]:
from xgboost import XGBRegressor

## Load data

In [ ]:
boston = load_boston()

# Initializing the dataframe
data = pd.DataFrame(boston.data)

#Adding the feature names to the dataframe
data.columns = boston.feature_names
data.head()

In [ ]:
#Adding target variable to dataframe
data['PRICE'] = boston.target 

In [ ]:
# Spliting target variable and independent variables
X = data.drop(['PRICE'], axis = 1)
y = data['PRICE']

In [ ]:
# Matrix with feature transformation
# X = pd.read_csv("newBoston.csv")

In [ ]:
# Splitting to training and testing data
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 4)

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Optimization function

In [ ]:
def opt_bas(n_estimators, max_depth, eta):
    
    # parameters
    hyperparameters = {
        "n_estimators" : int(n_estimators),
        "max_depth" : int(max_depth),
        "eta" : eta,
        "objective": "reg:squarederror",
        "random_state": 1,
    }
    
    # fit and predict
    model = XGBRegressor(**hyperparameters).fit(X_train, y_train)

    y_pred = model.predict(X_test)
    
    # get score
    metric = np.sqrt(np.mean(np.square((y_test - y_pred)/y_test)))*100
    
    return -metric

## Run optimization

In [ ]:
class newJSONLogger(JSONLogger):

      def __init__(self, path):
            self._path=None
            super(JSONLogger, self).__init__()
            self._path = path if path[-5:] == ".json" else path + ".json"

In [23]:
# Bounded region of parameter space
pbounds = {'n_estimators': (200, 300), 'max_depth': (25, 30), 'eta': (0.008, 0.6)}

# Bayes optimizer instantiation
optimizer = BayesianOptimization(f=opt_bas, 
                                 pbounds=pbounds, 
                                 random_state=1, verbose=2, 
                                )

# keep data
log_path = Path().resolve() / "Logs" / "Boston_rmspe_leo.json"
logger = newJSONLogger(path = str(log_path))
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

In [ ]:
optimizer.maximize(init_points=5, n_iter=500)